In [5]:
import os
import shutil
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
src_directory = '/Users/joanna/Computing_studies/CV/wk_proj3/dtd/images'
train_dir = '/Users/joanna/Computing_studies/CV/wk_proj3/train'
val_dir = '/Users/joanna/Computing_studies/CV/wk_proj3/validation'
test_dir = '/Users/joanna/Computing_studies/CV/wk_proj3/test'

# Create training, validation, test directories and distribute files
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

for class_name in os.listdir(src_directory):
    class_dir = os.path.join(src_directory, class_name)
    if os.path.isdir(class_dir):
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        train_files, val_test_files = train_test_split(files, test_size=0.3, random_state=42)
        val_files, test_files = train_test_split(val_test_files, test_size=0.5, random_state=42)

        for f in train_files:
            shutil.copy(f, os.path.join(train_dir, class_name))
        for f in val_files:
            shutil.copy(f, os.path.join(val_dir, class_name))
        for f in test_files:
            shutil.copy(f, os.path.join(test_dir, class_name))

# Setup data generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Load and configure the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(os.listdir(train_dir)), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, steps_per_epoch=100, epochs=20, validation_data=validation_generator, validation_steps=50)

# Evaluate the model
results = model.evaluate(test_generator)
print("Test Accuracy: ", results[1])


Found 3948 images belonging to 47 classes.
Found 846 images belonging to 47 classes.
Found 846 images belonging to 47 classes.
Epoch 1/20


/Users/joanna/miniconda3/envs/cv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - accuracy: 0.0717 - loss: 3.8070 - val_accuracy: 0.2730 - val_loss: 2.8752
Epoch 2/20


2024-06-05 12:23:17.782496: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/joanna/miniconda3/envs/cv/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


 24/100 ━━━━━━━━━━━━━━━━━━━━ 3:14 3s/step - accuracy: 0.1925 - loss: 3.0976

...

2024-06-05 12:32:52.571163: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 321s 3s/step - accuracy: 0.3271 - loss: 2.5419 - val_accuracy: 0.4291 - val_loss: 2.2277
Epoch 6/20


2024-06-05 12:38:13.706015: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


 10/100 ━━━━━━━━━━━━━━━━━━━━ 3:54 3s/step - accuracy: 0.3550 - loss: 2.4884

KeyboardInterrupt: 